In [1]:
import os
import h5py
import numpy as np

In [2]:
path = "first_100_epochs/model_epoch_100.h5"

In [8]:
f = h5py.File(path)

In [24]:
try:
    if len(f.attrs.items()):
        print("Parameter file: {}".format(path))
        print("Extracting model parameters to parameter dictionary ...")
        
except ValueError as ve:
    print(ve)
          
if len(f.items())==0:
    raise ValueError('No Layers with paramters found')
else:
    weights = {}
        
    # each layer has a group
    for pp in f.items():
        model_layers = pp
        break
        
        
    print(model_layers)
    # each layer has a group
    for layer, group in model_layers:

        # in the group the keys are the members: either the layer bias and/or parameters
        for p_name in group.keys():
            param = group[p_name]
            print(p_name, param)

            # if the group has 0 members then the layer has no biases or parameters
            if len(param) == 0:
                weights[layer, None] = None

            else:
                # for each parameter in the layer output to dictionary
                for k_name in param.keys():
                    print(p_name, param)
                    weights[layer, k_name] = param[k_name][()].tolist()

Parameter file: first_100_epochs/model_epoch_100.h5
Extracting model parameters to parameter dictionary ...
('model_weights', <HDF5 group "/model_weights" (23 members)>)


ValueError: too many values to unpack (expected 2)

In [61]:
weights = {}
# each layer has a group
count = 0
for layer, group in f.items():
    if count == 1:
        break
    count += 1
    # print(layer, group)
    
    for l, g in group.items():
        print(l, g)
        
        for p_name in g.keys():
            param = g[p_name]
            # print(p_name, param)
            
            # if the group has 0 members then the layer has no biases or parameters
            if len(param) == 0:
                weights[l, None] = None
            else:
                # for each parameter in the layer output to dictionary
                for k_name in param.keys():
                    # print(l, "###########", k_name)
                    # print(p_name,"####", k_name, param[k_name][()])
                    weights[l, k_name] = param[k_name][()].tolist()

add <HDF5 group "/model_weights/add" (0 members)>
add_1 <HDF5 group "/model_weights/add_1" (0 members)>
add_2 <HDF5 group "/model_weights/add_2" (0 members)>
add_3 <HDF5 group "/model_weights/add_3" (0 members)>
batch_normalization <HDF5 group "/model_weights/batch_normalization" (1 members)>
conv2_plus1d <HDF5 group "/model_weights/conv2_plus1d" (2 members)>
dense_3 <HDF5 group "/model_weights/dense_3" (1 members)>
flatten <HDF5 group "/model_weights/flatten" (0 members)>
global_average_pooling3d <HDF5 group "/model_weights/global_average_pooling3d" (0 members)>
input_2 <HDF5 group "/model_weights/input_2" (0 members)>
project <HDF5 group "/model_weights/project" (2 members)>
project_1 <HDF5 group "/model_weights/project_1" (2 members)>
project_2 <HDF5 group "/model_weights/project_2" (2 members)>
re_lu <HDF5 group "/model_weights/re_lu" (0 members)>
residual_main <HDF5 group "/model_weights/residual_main" (6 members)>
residual_main_1 <HDF5 group "/model_weights/residual_main_1" (6 me

In [48]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten 
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from keras import layers
import einops
from tensorflow import keras
from keras.models import Model

class Conv2Plus1D(keras.layers.Layer):
    def __init__(self, filters, kernel_size, padding, **kwargs):
        """
          A sequence of convolutional layers that first apply the convolution operation over the
          spatial dimensions, and then the temporal dimension. 
        """
        super(Conv2Plus1D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.seq = keras.Sequential([  
            # Spatial decomposition
            layers.Conv3D(filters=self.filters,
                          kernel_size=(1, self.kernel_size[1], self.kernel_size[2]),
                          padding=self.padding),
            # Temporal decomposition
            layers.Conv3D(filters=self.filters, 
                          kernel_size=(self.kernel_size[0], 1, 1),
                          padding=self.padding)
            ])

    def call(self, x):
        return self.seq(x)

    def get_config(self):
        config = super().get_config()
        config.update({
            "filters": self.filters,
            "kernel_size": self.kernel_size,
            "padding": self.padding,
        })
        return config


class ResidualMain(keras.layers.Layer):
    """
    Residual block of the model with convolution, layer normalization, and the
    activation function, ReLU.
    """
    def __init__(self, filters, kernel_size):
        super().__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.seq = keras.Sequential([
            Conv2Plus1D(filters=filters,
                        kernel_size=kernel_size,
                        padding='same'),
            layers.LayerNormalization(),
            layers.ReLU(),
            Conv2Plus1D(filters=filters, 
                        kernel_size=kernel_size,
                        padding='same'),
            layers.LayerNormalization()
        ])

    def call(self, x):
        return self.seq(x)

    def get_config(self):
        config = super().get_config()
        config.update({
            "filters": self.filters,
            "kernel_size": self.kernel_size,
        })
        return config

class Project(keras.layers.Layer):
    """
    Project certain dimensions of the tensor as the data is passed through different 
    sized filters and downsampled. 
    """
    def __init__(self, units, **kwargs):
        super(Project, self).__init__(**kwargs)
        self.units = units
        self.seq = keras.Sequential([
            layers.Dense(self.units),
            layers.LayerNormalization()
        ])

    def call(self, x):
        return self.seq(x)

    def get_config(self):
        config = super().get_config()
        config.update({
            "units": self.units,
        })
        return config

def add_residual_block(input, filters, kernel_size):
    """
    Add residual blocks to the model. If the last dimensions of the input data
    and filter size does not match, project it such that last dimension matches.
    """
    out = ResidualMain(filters, 
                     kernel_size)(input)

    res = input
    # Using the Keras functional APIs, project the last dimension of the tensor to
    # match the new filter size
    if out.shape[-1] != input.shape[-1]:
        res = Project(out.shape[-1])(res)

    return layers.add([res, out])


class ResizeVideo(keras.layers.Layer):
    def __init__(self, height, width, **kwargs):
        super(ResizeVideo, self).__init__(**kwargs)
        self.height = height
        self.width = width

    def call(self, video):
        """
          Use the einops library to resize the tensor.  

          Args:
            video: Tensor representation of the video, in the form of a set of frames.

          Return:
            A downsampled size of the video according to the new height and width it should be resized to.
        """
        # b stands for batch size, t stands for time, h stands for height, 
        # w stands for width, and c stands for the number of channels.
        old_shape = einops.parse_shape(video, 'b t h w c')
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
        resizing_layer = keras.layers.Resizing(self.height, self.width)
        images = resizing_layer(images)
        videos = einops.rearrange(
            images, '(b t) h w c -> b t h w c',
            t = old_shape['t'])
        return videos

    def get_config(self):
        config = super().get_config()
        config.update({
            "height": self.height,
            "width": self.width,
            "resizing_layer": self.resizing_layer,
        })
        return config


input_shape = (None, 30, 224, 224, 3)
HEIGHT = input_shape[2]
WIDTH = input_shape[3]
NUM_CLASSES = 9

input = layers.Input(shape=(input_shape[1:]))
x = input

x = Conv2Plus1D(filters=16, kernel_size=(3, 7, 7), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = ResizeVideo(HEIGHT // 2, WIDTH // 2)(x)

# Block 1
x = add_residual_block(x, 16, (3, 3, 3))
x = ResizeVideo(HEIGHT // 4, WIDTH // 4)(x)

# Block 2
x = add_residual_block(x, 32, (3, 3, 3))
x = ResizeVideo(HEIGHT // 8, WIDTH // 8)(x)

# Block 3
x = add_residual_block(x, 64, (3, 3, 3))
x = ResizeVideo(HEIGHT // 16, WIDTH // 16)(x)

# Block 4
x = add_residual_block(x, 128, (3, 3, 3))

x = layers.GlobalAveragePooling3D()(x)
x = layers.Flatten()(x)
x = layers.Dense(NUM_CLASSES)(x)

model = keras.Model(input, x)

In [49]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 30, 224, 22  0           []                               
                                4, 3)]                                                            
                                                                                                  
 conv2_plus1d_9 (Conv2Plus1D)   (None, 30, 224, 224  3152        ['input_2[0][0]']                
                                , 16)                                                             
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 30, 224, 224  64         ['conv2_plus1d_9[0][0]']         
 rmalization)                   , 16)                                                         

In [44]:

# Compile the model
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001), metrics=['accuracy'])


NameError: name 'model' is not defined

In [51]:
# Iterate through layers and set the weights
for layer_name, layer_weights in weights.items():
    layer = model.get_layer(name=layer_name)
    if layer is not None:
        layer.set_weights(layer_weights)

ValueError: No such layer: ('batch_normalization', 'beta:0'). Existing layers are: ['input_2', 'conv2_plus1d_9', 'batch_normalization_1', 're_lu_5', 'resize_video_4', 'residual_main_4', 'add_4', 'resize_video_5', 'project_3', 'residual_main_5', 'add_5', 'resize_video_6', 'project_4', 'residual_main_6', 'add_6', 'resize_video_7', 'project_5', 'residual_main_7', 'add_7', 'global_average_pooling3d_1', 'flatten_1', 'dense_6'].

In [52]:
weights.keys()

dict_keys([('batch_normalization', 'beta:0'), ('batch_normalization', 'gamma:0'), ('batch_normalization', 'moving_mean:0'), ('batch_normalization', 'moving_variance:0'), ('conv2_plus1d', 'bias:0'), ('conv2_plus1d', 'kernel:0'), ('dense_3', 'bias:0'), ('dense_3', 'kernel:0'), ('project', 'bias:0'), ('project', 'kernel:0'), ('project', 'beta:0'), ('project', 'gamma:0'), ('project_1', 'bias:0'), ('project_1', 'kernel:0'), ('project_1', 'beta:0'), ('project_1', 'gamma:0'), ('project_2', 'bias:0'), ('project_2', 'kernel:0'), ('project_2', 'beta:0'), ('project_2', 'gamma:0'), ('residual_main', 'bias:0'), ('residual_main', 'kernel:0'), ('residual_main', 'beta:0'), ('residual_main', 'gamma:0'), ('residual_main_1', 'bias:0'), ('residual_main_1', 'kernel:0'), ('residual_main_1', 'beta:0'), ('residual_main_1', 'gamma:0'), ('residual_main_2', 'bias:0'), ('residual_main_2', 'kernel:0'), ('residual_main_2', 'beta:0'), ('residual_main_2', 'gamma:0'), ('residual_main_3', 'bias:0'), ('residual_main_3',

In [58]:
for layer in model.layers:
    print(layer.name)

input_2
conv2_plus1d_9
batch_normalization_1
re_lu_5
resize_video_4
residual_main_4
add_4
resize_video_5
project_3
residual_main_5
add_5
resize_video_6
project_4
residual_main_6
add_6
resize_video_7
project_5
residual_main_7
add_7
global_average_pooling3d_1
flatten_1
dense_6
